In [1]:
#installing kaggle library
! pip install kaggle


In [2]:
# configuring path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
#api command
!kaggle datasets download -d bwandowando/3-million-instagram-google-store-reviews


Dataset URL: https://www.kaggle.com/datasets/bwandowando/3-million-instagram-google-store-reviews
License(s): CC0-1.0
 90% 192M/214M [00:00<00:00, 279MB/s]
100% 214M/214M [00:00<00:00, 339MB/s]


In [4]:
#extracting zip file
from zipfile import ZipFile
dataset = '/content/3-million-instagram-google-store-reviews.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('extracted')

extracted


In [5]:
#required libraries
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [8]:
instagram_data=pd.read_csv('/content/INSTAGRAM_REVIEWS.csv',encoding= 'utf-8')

In [9]:
instagram_data.shape

(3079939, 9)

In [10]:
instagram_data.head()

,Unnamed: 0,review_id,pseudo_author_id,author_name,review_text,review_rating,review_likes,author_app_version,review_timestamp
0,0,ff25650a-a9e1-4c02-a8e8-0c85b9ee1dc9,127325963209728431078,B ****or,Used this app for years. Was a great way to es...,1,0,216.1.0.21.137,2022-01-05 23:37:15
1,1,7e1840fe-4307-4c97-974c-437ec6c64e60,148177168797697591284,مس*******لو,khayemalo sag bgad,5,0,155.0.0.37.107,2022-01-05 23:39:29
2,2,3c8ba6fc-3009-4b1c-9a24-9ea635799a9a,162050472706594017332,Li********on,Lovely,5,0,NaN,2022-01-05 23:39:32
3,3,47c54668-dbac-4f8f-a98a-3ede00a103ea,264067006274581079256,Fa******Zx,best,5,0,NaN,2022-01-05 23:39:50
4,4,464a6a51-9a9f-44b0-80b5-eeea24adcd57,224172402296862948810,fr***************67,Super,5,0,216.0.0.20.137,2022-01-05 23:40:08


In [11]:
instagram_data.isnull().sum()

,0
Unnamed: 0,0
review_id,0
pseudo_author_id,0
author_name,0
review_text,128
review_rating,0
review_likes,0
author_app_version,765552
review_timestamp,0


In [12]:
instagram_data.dropna(subset=['review_text'], inplace=True)  # Drop missing reviews

instagram_data['author_app_version'] = instagram_data['author_app_version'].fillna('unknown') # Fill Nan Values

In [13]:
instagram_data.isnull().sum()


,0
Unnamed: 0,0
review_id,0
pseudo_author_id,0
author_name,0
review_text,0
review_rating,0
review_likes,0
author_app_version,0
review_timestamp,0


In [14]:
instagram_data.shape

(3079811, 9)

In [15]:
instagram_data.head()

,Unnamed: 0,review_id,pseudo_author_id,author_name,review_text,review_rating,review_likes,author_app_version,review_timestamp
0,0,ff25650a-a9e1-4c02-a8e8-0c85b9ee1dc9,127325963209728431078,B ****or,Used this app for years. Was a great way to es...,1,0,216.1.0.21.137,2022-01-05 23:37:15
1,1,7e1840fe-4307-4c97-974c-437ec6c64e60,148177168797697591284,مس*******لو,khayemalo sag bgad,5,0,155.0.0.37.107,2022-01-05 23:39:29
2,2,3c8ba6fc-3009-4b1c-9a24-9ea635799a9a,162050472706594017332,Li********on,Lovely,5,0,unknown,2022-01-05 23:39:32
3,3,47c54668-dbac-4f8f-a98a-3ede00a103ea,264067006274581079256,Fa******Zx,best,5,0,unknown,2022-01-05 23:39:50
4,4,464a6a51-9a9f-44b0-80b5-eeea24adcd57,224172402296862948810,fr***************67,Super,5,0,216.0.0.20.137,2022-01-05 23:40:08


In [16]:
instagram_data['review_rating'].value_counts()

,count
review_rating,
5,1963214
1,589844
4,246602
3,155200
2,124951


In [22]:
def label_sentiment(rating):
    if rating >= 4:
        return 'positive review'
    elif rating == 3:
        return 'neutral review'
    else:
        return 'negative review'

In [21]:
instagram_data['sentiment'] = instagram_data['review_rating'].apply(label_sentiment)
print(instagram_data['sentiment'].value_counts())

sentiment
positive review    2209816
negative review     714795
neutral review      155200
Name: count, dtype: int64


In [18]:
port_stem=PorterStemmer()
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [23]:
instagram_data['stemmed_content']=instagram_data['review_text'].apply(stemming)

In [24]:
instagram_data.head()

,Unnamed: 0,review_id,pseudo_author_id,author_name,review_text,review_rating,review_likes,author_app_version,review_timestamp,sentiment,stemmed_content
0,0,ff25650a-a9e1-4c02-a8e8-0c85b9ee1dc9,127325963209728431078,B ****or,Used this app for years. Was a great way to es...,1,0,216.1.0.21.137,2022-01-05 23:37:15,negative review,use app year great way escap overli polit envi...
1,1,7e1840fe-4307-4c97-974c-437ec6c64e60,148177168797697591284,مس*******لو,khayemalo sag bgad,5,0,155.0.0.37.107,2022-01-05 23:39:29,positive review,khayemalo sag bgad
2,2,3c8ba6fc-3009-4b1c-9a24-9ea635799a9a,162050472706594017332,Li********on,Lovely,5,0,unknown,2022-01-05 23:39:32,positive review,love
3,3,47c54668-dbac-4f8f-a98a-3ede00a103ea,264067006274581079256,Fa******Zx,best,5,0,unknown,2022-01-05 23:39:50,positive review,best
4,4,464a6a51-9a9f-44b0-80b5-eeea24adcd57,224172402296862948810,fr***************67,Super,5,0,216.0.0.20.137,2022-01-05 23:40:08,positive review,super


In [30]:
#separating data and label
X=instagram_data['stemmed_content'].values
Y=instagram_data['review_rating'].values

In [31]:
print(X)

['use app year great way escap overli polit environ past decad ig felt necessari feed fill polit social caus delet app need life black live matter scam'
 'khayemalo sag bgad' 'love' ...
 'thank team much bring back old remix option still complaint last night tri remix video two way st without tri remix option show video nd tri remix option show video way video upload success lip match also music video sync pleas fix problem high prioriti'
 'lack featur' 'neya jii']


In [32]:
print(Y)

[1 5 5 ... 3 2 1]


In [33]:
#spliting data into test and training data
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [34]:
print(X.shape,X_train.shape,X_test.shape)

(3079811,) (2463848,) (615963,)


In [37]:
#converting textual data to numerical data
vectorizer=TfidfVectorizer()

X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)